# PROBLEM A5

Build and train a neural network model using the Sunspots.csv dataset.
Use MAE as the metrics of your neural network model.
We provided code for normalizing the data. Please do not change the code.
Do not use lambda layers in your model.

The dataset used in this problem is downloaded from kaggle.com/robervalt/sunspots

Desired MAE < 0.15 on the normalized dataset.

Setting the imports

In [ ]:
!pip install numpy==1.24.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 88.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.


In [ ]:
!pip install pandas==2.0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 99.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 37.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is incompatible.


In [ ]:
!pip install Pillow==10.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 35.7 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0


In [ ]:
!pip install scipy==1.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 17.3 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.3
    Uninstalling scipy-1.11.3:
      Successfully uninstalled scipy-1.11.3


In [ ]:
!pip install tensorflow==2.13.0

In [ ]:
!pip install tensorflow-datasets==4.9.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 13.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.9.3
    Uninstalling tensorflow-datasets-4.9.3:
      Successfully uninstalled tensorflow-datasets-4.9.3


Import

In [ ]:
import tensorflow as tf
import pandas as pd
import PIL as pil
import scipy as sc
import numpy as np
import tensorflow_datasets as tfds
import urllib
import csv

Checking Library version

In [ ]:
print(np.__version__)
print(pd.__version__)
print(pil.__version__)
print(sc.__version__)
print(tf.__version__)
print(tfds.__version__)

1.24.3
2.0.3
10.0.0
1.10.1
2.13.0
4.9.2


Getting The Data

In [ ]:
data_url = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/sunspots.csv'
urllib.request.urlretrieve(data_url, 'sunspots.csv')

('sunspots.csv', <http.client.HTTPMessage at 0x794951ac2410>)

In [ ]:
time_step = []
sunspots = []

In [ ]:
with open('sunspots.csv') as csvfile:
  reader = csv.reader(csvfile, delimiter=',')
  # skip the header of the column
  next(reader)
  for row in reader:
    sunspots.append(float(row[2])) # get the sunspots
    time_step.append(int(row[0])) # get the time and turn it into int, this will turn the dates into number, like 1,2,3,4

In [ ]:
sunspots[0:5]

[96.7, 104.3, 116.7, 92.8, 141.7]

In [ ]:
time_step[0:5]

[0.0, 1.0, 2.0, 3.0, 4.0]

In [ ]:
# Turn them both into a numpy array
series = np.array(sunspots)
time = np.array(time_step)
# this will give as a 2 dimensional array
series.shape

(9795,)

Normalize the Data and create the split

In [ ]:
# Normalize by first getting the max and minimum value of a series
min = np.min(series)
max = np.max(series)

In [ ]:
# Do the normalization
series -= min
series /= max

In [ ]:
# define the split, how many train data and test data
split_time=3000

In [ ]:
train_series = series[:split_time]
train_time = time[:split_time]

test_series = time[split_time:]
test_time = time[split_time:]

Create the windowed dataset

In [ ]:
# some variable to set the windowed dataset
window_size=30
batch_size=32
shuffle_buffer_size=1000

In [ ]:
def windowed_dataset(series, window_size, batch_size,  shuffle_buffer):
  # turn it into a 3 dimension array from 2 dimension, like [[1,2,3,4]] to [[1],[2],[3]] that is needed to create the windowed dataset
  series = tf.expand_dims(series, axis=-1) #-1 means the rows
  print(series[0:5])
  dataset = tf.data.Dataset.from_tensor_slices(series) # create a tf.data.Dataset object from an array (tensor slices)
  print(dataset)
  dataset = dataset.window(window_size+1, shift=1, drop_remainder=True)

  dataset = dataset.flat_map(lambda w: w.batch(window_size+1))

  dataset = dataset.shuffle(shuffle_buffer_size)

  dataset = dataset.map(lambda w: (w[:-1],w[-1]))

  return dataset.batch(batch_size).prefetch(1)

1. tf.expand_dims(series, axis=-1): This line takes the input time series data series and adds a new axis at the end of the data. This reshapes the data to have an additional dimension, which is often required when working with TensorFlow.

2. tf.data.Dataset.from_tensor_slices(series): This creates a TensorFlow Dataset from the input data series. A dataset in TensorFlow is a sequence of elements that can be iterated over.

3. ds.window(window_size + 1, shift=1, drop_remainder=True): This creates a "windowed" dataset. It splits the data into overlapping windows of size window_size + 1 (the +1 is used to include both the input and target values for a prediction). The shift=1 parameter specifies that the windows should slide by one element at a time. drop_remainder=True means that any window that doesn't have enough elements to form a complete window of size window_size + 1 will be dropped.

4. ds.flat_map(lambda w: w.batch(window_size + 1)): The flat_map operation is used to flatten the windows created in the previous step. Each window is converted into a batch of window_size + 1 elements. This prepares the data for creating input and target pairs for prediction.

5. ds.shuffle(shuffle_buffer): This shuffles the dataset. The parameter shuffle_buffer determines how many elements in the dataset should be kept in a buffer, and TensorFlow randomly samples elements from this buffer to create a shuffled dataset. Shuffling is often done to introduce randomness during training, which can help the model generalize better.

6. ds.map(lambda w: (w[:-1], w[1:])): This maps each element in the dataset to a tuple containing two parts. The first part, w[:-1], represents the input sequence, excluding the last element. The second part, w[1:], represents the target sequence, excluding the first element. This is a common approach for creating input-target pairs for sequence prediction tasks.

7. return ds.batch(batch_size).prefetch(1): Finally, the dataset is batched into batches of size batch_size. Batching is a common practice in machine learning to process multiple examples simultaneously, which can improve training efficiency. The prefetch(1) operation helps to improve data loading performance by prefetching the next batch of data, allowing the GPU or CPU to work on the current batch without waiting for data loading1

In [ ]:
train_set=windowed_dataset(train_series, window_size=window_size,batch_size=batch_size, shuffle_buffer=shuffle_buffer_size)

tf.Tensor(
[[0.24284279]
 [0.26192868]
 [0.29306881]
 [0.23304872]
 [0.35585133]], shape=(5, 1), dtype=float64)
<_TensorSliceDataset element_spec=TensorSpec(shape=(1,), dtype=tf.float64, name=None)>


Create the model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(window_size, kernel_size=1, input_shape=[None,1], activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16, activation='relu'),
    # YOUR CODE HERE.
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(loss='mae', optimizer='adam', metrics=['mae'])

In [ ]:
model.fit(train_set, epochs = 500)

Epoch 1/500
93/93 [==============================] - 13s 12ms/step - loss: 0.0905 - mae: 0.0905
Epoch 2/500
93/93 [==============================] - 1s 7ms/step - loss: 0.0564 - mae: 0.0564
Epoch 3/500
93/93 [==============================] - 1s 8ms/step - loss: 0.0538 - mae: 0.0538
Epoch 4/500
93/93 [==============================] - 1s 8ms/step - loss: 0.0524 - mae: 0.0524
Epoch 5/500
93/93 [==============================] - 1s 7ms/step - loss: 0.0519 - mae: 0.0519
Epoch 6/500
93/93 [==============================] - 1s 8ms/step - loss: 0.0514 - mae: 0.0514
Epoch 7/500
93/93 [==============================] - 1s 12ms/step - loss: 0.0511 - mae: 0.0511
Epoch 8/500
93/93 [==============================] - 1s 13ms/step - loss: 0.0509 - mae: 0.0509
Epoch 9/500
93/93 [==============================] - 1s 11ms/step - loss: 0.0506 - mae: 0.0506
Epoch 10/500
93/93 [==============================] - 1s 8ms/step - loss: 0.0505 - mae: 0.0505
Epoch 11/500
93/93 [==============================] - 